In [ ]:

#!/usr/bin/env python3
# Authors: Dinesh M, 
#          Ragothaman M. Yennamalli, SASTRA Deemed to be University
# Last Modified: 2025-05-11
# Description:
#   This python scripts implements various deep learning models for PANN classification in Tamil poetry.
#   It loads preprocessed embeddings and PANN labels from structured datasets.
#   Multiple architectures like RNN, FFNN, CNN, LSTM, BiLSTM, MLP, GRU, and Transformer are trained and evaluated.
#   Model performance is assessed using metrics like Precision, Recall, and F1-score.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EmbeddingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class RNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_layers=1, dropout=0.3):
        super().__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers,
                          batch_first=True, nonlinearity='tanh')
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  
        rnn_out, _ = self.rnn(x)
        out = rnn_out[:, -1, :]  
        out = self.dropout(out)
        return self.classifier(out)

class EarlyStopping:
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

def train_model(model, dataloader, optimizer, criterion):
    model.train()
    for X_batch, y_batch in tqdm(dataloader, desc="Training"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in tqdm(dataloader, desc="Evaluating"):
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.numpy())
    return all_labels, all_preds

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Embedding'] = df['Embedding'].apply(ast.literal_eval)
    X = np.array(df['Embedding'].tolist(), dtype=np.float32)
    y = df['PANN_LABEL'].values
    return X, y

def run_rnn(train_file, val_file, test_file):
    X_train, y_train = load_data(train_file)
    X_val, y_val = load_data(val_file)
    X_test, y_test = load_data(test_file)

    label_encoder = LabelEncoder()
    y_train_enc = label_encoder.fit_transform(y_train)
    y_val_enc = label_encoder.transform(y_val)
    y_test_enc = label_encoder.transform(y_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    train_loader = DataLoader(EmbeddingDataset(X_train, y_train_enc), batch_size=32, shuffle=True)
    val_loader = DataLoader(EmbeddingDataset(X_val, y_val_enc), batch_size=32)
    test_loader = DataLoader(EmbeddingDataset(X_test, y_test_enc), batch_size=32)

    input_dim = X_train.shape[1]
    num_classes = len(label_encoder.classes_)
    model = RNNClassifier(input_dim=input_dim, hidden_dim=128, num_classes=num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    early_stopper = EarlyStopping(patience=3)

    for epoch in range(20):
        print(f"\n Epoch {epoch+1}")
        train_model(model, train_loader, optimizer, criterion)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f" Validation Loss: {val_loss:.4f}")

        early_stopper(val_loss)
        if early_stopper.early_stop:
            print(" Early stopping triggered!")
            break

    y_true, y_pred = evaluate_model(model, test_loader)
    print("\n Classification Report (Test):")
    print(classification_report(y_true, y_pred, zero_division=0))
    print(" Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print(f" Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f" Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" Recall:    {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" F1 Score:  {f1_score(y_true, y_pred, average='weighted'):.4f}")

    torch.save(model.state_dict(), "rnn_model.pt")
    print("💾 Model saved as rnn_model.pt")

train_file = "path_to_your_directory"
val_file = "path_to_your_directory"
test_file = "path_to_your_directory"

run_rnn(train_file, val_file, test_file)



📚 Epoch 1


Training: 100%|██████████| 448/448 [00:01<00:00, 281.15it/s]


📉 Validation Loss: 1.7816

📚 Epoch 2


Training: 100%|██████████| 448/448 [00:00<00:00, 461.63it/s]


📉 Validation Loss: 1.4488

📚 Epoch 3


...

Training: 100%|██████████| 448/448 [00:00<00:00, 454.12it/s]


📉 Validation Loss: 0.7519


Evaluating: 100%|██████████| 150/150 [00:00<00:00, 983.76it/s]


📊 Classification Report (Test):
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       141
           1       0.39      0.21      0.27       134
           2       0.73      0.78      0.75       139
           3       0.51      0.36      0.42       124
           4       0.59      0.50      0.54       115
           5       0.99      1.00      1.00       135
           6       0.62      0.54      0.58       113
           7       0.85      0.92      0.88       114
           8       0.80      0.89      0.84       128
           9       0.66      0.70      0.68       146
          10       0.68      0.69      0.68       134
          11       1.00      1.00      1.00       149
          12       0.99      1.00      0.99       149
          13       0.87      0.95      0.91       139
          14       0.64      0.49      0.55       134
          15       0.64      0.56      0.60       151
          16       0.62      0.62      0.62     

✅ F1 Score:  0.8100
💾 Model saved as rnn_model.pt


In [ ]:
import pandas as pd
import numpy as np
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EmbeddingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self): return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, num_filters=100, kernel_sizes=[2, 3, 4], dropout=0.5):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=k)
            for k in kernel_sizes
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

    def forward(self, x):
        x = x.unsqueeze(1) 
        x = [torch.relu(conv(x)).squeeze(2) for conv in self.convs]
        x = [torch.max(feature_map, dim=2)[0] if feature_map.dim() == 3 else feature_map for feature_map in x]
        x = torch.cat(x, dim=1)
        x = self.dropout(x)
        return self.fc(x)

class EarlyStopping:
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

def train_model(model, dataloader, optimizer, criterion):
    model.train()
    for X_batch, y_batch in tqdm(dataloader, desc="Training"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in tqdm(dataloader, desc="Evaluating"):
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.numpy())
    return all_labels, all_preds

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Embedding'] = df['Embedding'].apply(ast.literal_eval)
    X = np.array(df['Embedding'].tolist(), dtype=np.float32)
    y = df['PANN_LABEL'].values
    return X, y

def run_cnn(train_file, val_file, test_file):
    X_train, y_train = load_data(train_file)
    X_val, y_val = load_data(val_file)
    X_test, y_test = load_data(test_file)

    label_encoder = LabelEncoder()
    y_train_enc = label_encoder.fit_transform(y_train)
    y_val_enc = label_encoder.transform(y_val)
    y_test_enc = label_encoder.transform(y_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    train_loader = DataLoader(EmbeddingDataset(X_train, y_train_enc), batch_size=32, shuffle=True)
    val_loader = DataLoader(EmbeddingDataset(X_val, y_val_enc), batch_size=32)
    test_loader = DataLoader(EmbeddingDataset(X_test, y_test_enc), batch_size=32)

    input_dim = X_train.shape[1]
    num_classes = len(label_encoder.classes_)
    model = CNNClassifier(input_dim=input_dim, num_classes=num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    early_stopper = EarlyStopping(patience=3)

    for epoch in range(20):
        print(f"\n Epoch {epoch+1}")
        train_model(model, train_loader, optimizer, criterion)

        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f" Validation Loss: {val_loss:.4f}")

        early_stopper(val_loss)
        if early_stopper.early_stop:
            print(" Early stopping triggered!")
            break

    y_true, y_pred = evaluate_model(model, test_loader)
    print("\n Classification Report (Test):")
    print(classification_report(y_true, y_pred, zero_division=0))
    print(" Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print(f" Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f" Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" Recall:    {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" F1 Score:  {f1_score(y_true, y_pred, average='weighted'):.4f}")

    torch.save(model.state_dict(), "cnn_model.pt")
    print("💾 Model saved as cnn_model.pt")

train_file = "path_to_your_directory"
val_file = "path_to_your_directory"
test_file = "path_to_your_directory"

run_cnn(train_file, val_file, test_file)



📚 Epoch 1


Training: 100%|██████████| 448/448 [00:02<00:00, 209.87it/s]


📉 Validation Loss: 3.7274

📚 Epoch 2


Training: 100%|██████████| 448/448 [00:01<00:00, 420.15it/s]


📉 Validation Loss: 3.6063

📚 Epoch 3


...

Training: 100%|██████████| 448/448 [00:01<00:00, 418.76it/s]


📉 Validation Loss: 3.3839


Evaluating: 100%|██████████| 150/150 [00:00<00:00, 612.35it/s]


📊 Classification Report (Test):
              precision    recall  f1-score   support

           0       0.17      0.13      0.15       141
           1       0.08      0.43      0.13       134
           2       0.27      0.05      0.08       139
           3       0.07      0.12      0.09       124
           4       0.00      0.00      0.00       115
           5       0.25      0.81      0.38       135
           6       0.00      0.00      0.00       113
           7       0.10      0.04      0.05       114
           8       0.12      0.04      0.06       128
           9       0.00      0.00      0.00       146
          10       0.03      0.02      0.02       134
          11       0.19      0.48      0.27       149
          12       0.15      0.16      0.16       149
          13       0.05      0.01      0.02       139
          14       0.00      0.00      0.00       134
          15       0.06      0.01      0.01       151
          16       0.00      0.00      0.00     


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import numpy as np
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EmbeddingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self): return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, nhead=4, num_layers=2, dim_feedforward=256, dropout=0.3):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  
        x = self.transformer(x)
        x = x.mean(dim=1) 
        return self.classifier(x)

class EarlyStopping:
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

def train_model(model, dataloader, optimizer, criterion):
    model.train()
    for X_batch, y_batch in tqdm(dataloader, desc="Training"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in tqdm(dataloader, desc="Evaluating"):
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.numpy())
    return all_labels, all_preds

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Embedding'] = df['Embedding'].apply(ast.literal_eval)
    X = np.array(df['Embedding'].tolist(), dtype=np.float32)
    y = df['PANN_LABEL'].values
    return X, y

def run_transformer(train_file, val_file, test_file):
    X_train, y_train = load_data(train_file)
    X_val, y_val = load_data(val_file)
    X_test, y_test = load_data(test_file)

    label_encoder = LabelEncoder()
    y_train_enc = label_encoder.fit_transform(y_train)
    y_val_enc = label_encoder.transform(y_val)
    y_test_enc = label_encoder.transform(y_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    train_loader = DataLoader(EmbeddingDataset(X_train, y_train_enc), batch_size=32, shuffle=True)
    val_loader = DataLoader(EmbeddingDataset(X_val, y_val_enc), batch_size=32)
    test_loader = DataLoader(EmbeddingDataset(X_test, y_test_enc), batch_size=32)

    input_dim = X_train.shape[1]
    num_classes = len(label_encoder.classes_)
    model = TransformerClassifier(input_dim=input_dim, num_classes=num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    early_stopper = EarlyStopping(patience=3)

    for epoch in range(20):
        print(f"\n Epoch {epoch+1}")
        train_model(model, train_loader, optimizer, criterion)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f" Validation Loss: {val_loss:.4f}")

        early_stopper(val_loss)
        if early_stopper.early_stop:
            print(" Early stopping triggered!")
            break

    y_true, y_pred = evaluate_model(model, test_loader)
    print("\n Classification Report (Test):")
    print(classification_report(y_true, y_pred, zero_division=0))
    print(" Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print(f" Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f" Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" Recall:    {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" F1 Score:  {f1_score(y_true, y_pred, average='weighted'):.4f}")

    torch.save(model.state_dict(), "transformer_model.pt")
    print(" Model saved as transformer_model.pt")

train_file = "path_to_your_directory"
val_file = "path_to_your_directory"
test_file = "path_to_your_directory"

run_transformer(train_file, val_file, test_file)



📚 Epoch 1


Training: 100%|██████████| 448/448 [00:02<00:00, 149.67it/s]


📉 Validation Loss: 1.9267

📚 Epoch 2


Training: 100%|██████████| 448/448 [00:03<00:00, 134.48it/s]


📉 Validation Loss: 1.7222

📚 Epoch 3


...

Training: 100%|██████████| 448/448 [00:02<00:00, 160.29it/s]


📉 Validation Loss: 1.0153


Evaluating: 100%|██████████| 150/150 [00:00<00:00, 818.75it/s]


📊 Classification Report (Test):
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       141
           1       0.21      0.21      0.21       134
           2       0.63      0.56      0.59       139
           3       0.22      0.15      0.18       124
           4       0.41      0.26      0.32       115
           5       0.99      1.00      1.00       135
           6       0.28      0.36      0.32       113
           7       0.76      0.84      0.80       114
           8       0.78      0.77      0.78       128
           9       0.67      0.68      0.68       146
          10       0.57      0.41      0.48       134
          11       0.98      1.00      0.99       149
          12       0.94      1.00      0.97       149
          13       0.73      0.96      0.83       139
          14       0.66      0.29      0.40       134
          15       0.51      0.57      0.54       151
          16       0.62      0.49      0.55     

[[140   0   0 ...   0   0   0]
 [  2  28   2 ...   1   1   7]
 [  0   2  78 ...   0   0  11]
 ...
 [  0   0   0 ... 151   0   0]
 [  0   0   0 ...   0 156   0]
 [  1   1   0 ...   0   0  98]]
✅ Accuracy:  0.7293
✅ Precision: 0.7254
✅ Recall:    0.7293
✅ F1 Score:  0.7206
💾 Model saved as transformer_model.pt


In [ ]:
import pandas as pd
import numpy as np
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EmbeddingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self): return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class GRUClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_layers=1, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        self.classifier = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1) 
        gru_out, _ = self.gru(x)
        out = gru_out[:, -1, :]  
        return self.classifier(out)

class EarlyStopping:
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

def train_model(model, dataloader, optimizer, criterion):
    model.train()
    for X_batch, y_batch in tqdm(dataloader, desc="Training"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in tqdm(dataloader, desc="Evaluating"):
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.numpy())
    return all_labels, all_preds

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Embedding'] = df['Embedding'].apply(ast.literal_eval)
    X = np.array(df['Embedding'].tolist(), dtype=np.float32)
    y = df['PANN_LABEL'].values
    return X, y

def run_gru(train_file, val_file, test_file):
   
    X_train, y_train = load_data(train_file)
    X_val, y_val = load_data(val_file)
    X_test, y_test = load_data(test_file)

    label_encoder = LabelEncoder()
    y_train_enc = label_encoder.fit_transform(y_train)
    y_val_enc = label_encoder.transform(y_val)
    y_test_enc = label_encoder.transform(y_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    train_loader = DataLoader(EmbeddingDataset(X_train, y_train_enc), batch_size=32, shuffle=True)
    val_loader = DataLoader(EmbeddingDataset(X_val, y_val_enc), batch_size=32)
    test_loader = DataLoader(EmbeddingDataset(X_test, y_test_enc), batch_size=32)

    input_dim = X_train.shape[1]
    num_classes = len(label_encoder.classes_)
    model = GRUClassifier(input_dim=input_dim, hidden_dim=128, num_classes=num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    early_stopper = EarlyStopping(patience=3)

    for epoch in range(20):
        print(f"\n📚 Epoch {epoch+1}")
        train_model(model, train_loader, optimizer, criterion)
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f"Validation Loss: {val_loss:.4f}")

        early_stopper(val_loss)
        if early_stopper.early_stop:
            print(" Early stopping triggered!")
            break

  
    y_true, y_pred = evaluate_model(model, test_loader)
    print("\n Classification Report (Test):")
    print(classification_report(y_true, y_pred, zero_division=0))
    print(" Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print(f" Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f" Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" Recall:    {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" F1 Score:  {f1_score(y_true, y_pred, average='weighted'):.4f}")

   
    torch.save(model.state_dict(), "gru_model.pt")
    print("Model saved as gru_model.pt")

train_file = "path_to_your_directory"
val_file = "path_to_your_directory"
test_file = "path_to_your_directory"

run_gru(train_file, val_file, test_file)


In [ ]:
import pandas as pd
import numpy as np
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EmbeddingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self): return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class FFNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, dropout=0.3):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, num_classes))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class EarlyStopping:
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

def train_model(model, dataloader, optimizer, criterion):
    model.train()
    for X_batch, y_batch in tqdm(dataloader, desc="Training"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in tqdm(dataloader, desc="Evaluating"):
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.numpy())
    return all_labels, all_preds

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Embedding'] = df['Embedding'].apply(ast.literal_eval)
    X = np.array(df['Embedding'].tolist(), dtype=np.float32)
    y = df['PANN_LABEL'].values
    return X, y


def run_ffnn(train_file, val_file, test_file):
    X_train, y_train = load_data(train_file)
    X_val, y_val = load_data(val_file)
    X_test, y_test = load_data(test_file)

    label_encoder = LabelEncoder()
    y_train_enc = label_encoder.fit_transform(y_train)
    y_val_enc = label_encoder.transform(y_val)
    y_test_enc = label_encoder.transform(y_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    train_loader = DataLoader(EmbeddingDataset(X_train, y_train_enc), batch_size=32, shuffle=True)
    val_loader = DataLoader(EmbeddingDataset(X_val, y_val_enc), batch_size=32)
    test_loader = DataLoader(EmbeddingDataset(X_test, y_test_enc), batch_size=32)

    input_dim = X_train.shape[1]
    num_classes = len(label_encoder.classes_)
    model = FFNNClassifier(input_dim=input_dim, hidden_dims=[256, 128], num_classes=num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    early_stopper = EarlyStopping(patience=3)
    for epoch in range(20):
        print(f"\n Epoch {epoch+1}")
        train_model(model, train_loader, optimizer, criterion)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f" Validation Loss: {val_loss:.4f}")


        early_stopper(val_loss)
        if early_stopper.early_stop:
            print("Early stopping triggered!")
            break

    y_true, y_pred = evaluate_model(model, test_loader)
    print("\n Classification Report (Test):")
    print(classification_report(y_true, y_pred, zero_division=0))
    print(" Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print(f" Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f" Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" Recall:    {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" F1 Score:  {f1_score(y_true, y_pred, average='weighted'):.4f}")

    torch.save(model.state_dict(), "ffnn_model.pt")
    print(" Model saved as ffnn_model.pt")

train_file = "path_to_your_directory"
val_file = "path_to_your_directory"
test_file = "path_to_your_directory"

run_ffnn(train_file, val_file, test_file)


In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import ast

torch.manual_seed(42)

train_file = "path_to_your_directory"
val_file = "path_to_your_directory"
test_file = "path_to_your_directory"

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Embedding'] = df['Embedding'].apply(ast.literal_eval)
    X_data = np.array(df['Embedding'].tolist(), dtype=np.float32)
    y_data = df['PANN_LABEL'].values
    return torch.tensor(X_data), torch.tensor(y_data)

X_train, y_train = load_data(train_file)
X_val, y_val = load_data(val_file)
X_test, y_test = load_data(test_file)

batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=256, num_layers=2, output_size=32, dropout_rate=0.5):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.layernorm = nn.LayerNorm(hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]  # Take last time-step output
        out = self.layernorm(out)
        out = self.relu(out)
        out = self.dropout(out)
        return self.fc(out)

input_size = X_train.shape[1]
output_size = len(torch.unique(y_train))
model = LSTMModel(input_size=input_size, output_size=output_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

num_epochs = 100
patience = 5
best_val_loss = float('inf')
epochs_without_improvement = 0
best_model_state = None
best_val_preds = None
best_val_labels = None

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

    model.eval()
    val_loss = 0
    y_val_true, y_val_pred = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            y_val_true.extend(labels.cpu().numpy())
            y_val_pred.extend(predicted.cpu().numpy())

    val_accuracy = np.mean(np.array(y_val_true) == np.array(y_val_pred))
    print(f"Validation Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        best_val_preds = y_val_pred.copy()
        best_val_labels = y_val_true.copy()
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    scheduler.step()

    if epochs_without_improvement >= patience:
        print("Early stopping triggered!")
        break

model.load_state_dict(best_model_state)

print("\n Best Validation Classification Report:")
print(classification_report(best_val_labels, best_val_preds, zero_division=0))

model.eval()
y_test_true, y_test_pred = [], []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_test_true.extend(labels.cpu().numpy())
        y_test_pred.extend(predicted.cpu().numpy())

print("\n Test Classification Report:")
print(classification_report(y_test_true, y_test_pred, zero_division=0))

print("\n Test Confusion Matrix:")
print(confusion_matrix(y_test_true, y_test_pred))

accuracy = accuracy_score(y_test_true, y_test_pred)
precision = precision_score(y_test_true, y_test_pred, average='weighted', zero_division=0)
recall = recall_score(y_test_true, y_test_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test_true, y_test_pred, average='weighted', zero_division=0)

print(f"\n Overall Test Accuracy:  {accuracy:.4f}")
print(f" Overall Precision:      {precision:.4f}")
print(f" Overall Recall:         {recall:.4f}")
print(f" Overall F1-Score:       {f1:.4f}")

torch.save(best_model_state, "best_lstm_model.pth")
print("\n Best LSTM model saved as 'best_lstm_model.pth'")


In [ ]:
import pandas as pd
import numpy as np
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EmbeddingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self): return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_layers=1, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        self.classifier = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_len=1, input_dim)
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]  # last timestep
        return self.classifier(out)

class EarlyStopping:
    def __init__(self, patience=2):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

def train_model(model, dataloader, optimizer, criterion):
    model.train()
    for X_batch, y_batch in tqdm(dataloader, desc="Training"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in tqdm(dataloader, desc="Evaluating"):
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.numpy())
    return all_labels, all_preds

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Embedding'] = df['Embedding'].apply(ast.literal_eval)
    X = np.array(df['Embedding'].tolist(), dtype=np.float32)
    y = df['PANN_LABEL'].values
    return X, y

def run_bilstm(train_file, val_file, test_file):
    X_train, y_train = load_data(train_file)
    X_val, y_val = load_data(val_file)
    X_test, y_test = load_data(test_file)

    label_encoder = LabelEncoder()
    y_train_enc = label_encoder.fit_transform(y_train)
    y_val_enc = label_encoder.transform(y_val)
    y_test_enc = label_encoder.transform(y_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    train_loader = DataLoader(EmbeddingDataset(X_train, y_train_enc), batch_size=32, shuffle=True)
    val_loader = DataLoader(EmbeddingDataset(X_val, y_val_enc), batch_size=32)
    test_loader = DataLoader(EmbeddingDataset(X_test, y_test_enc), batch_size=32)

    input_dim = X_train.shape[1]
    num_classes = len(label_encoder.classes_)
    model = BiLSTMClassifier(input_dim, hidden_dim=128, num_classes=num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    early_stopper = EarlyStopping(patience=3)

    for epoch in range(20):
        print(f"\n Epoch {epoch+1}")
        train_model(model, train_loader, optimizer, criterion)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f" Validation Loss: {val_loss:.4f}")

        early_stopper(val_loss)
        if early_stopper.early_stop:
            print("Early stopping triggered!")
            break

    y_true, y_pred = evaluate_model(model, test_loader)
    print("\nClassification Report (Test):")
    print(classification_report(y_true, y_pred, zero_division=0))
    print(" Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print(f" Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f" Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" Recall:    {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f" F1 Score:  {f1_score(y_true, y_pred, average='weighted'):.4f}")

    torch.save(model.state_dict(), "bilstm_model.pt")
    print(" Model saved as bilstm_model.pt")

train_file = "path_to_your_directory"
val_file = "path_to_your_directory"
test_file = "path_to_your_directory"

run_bilstm(train_file, val_file, test_file)


In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
import joblib

np.random.seed(42)

train_file = "path_to_your_directory"
val_file = "path_to_your_directory"
test_file = "path_to_your_directory"

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['Embedding'] = df['Embedding'].apply(ast.literal_eval)
    X_data = np.array(df['Embedding'].tolist(), dtype=np.float32)
    y_data = df['PANN_LABEL'].values
    return X_data, y_data

X_train, y_train = load_data(train_file)
X_val, y_val = load_data(val_file)
X_test, y_test = load_data(test_file)

label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_val_enc = label_encoder.transform(y_val)
y_test_enc = label_encoder.transform(y_test)

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

models = {
  "MLP Classifier": MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42),
}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_norm, y_train_enc)

    print("🔍 Validation Results:")
    val_preds = model.predict(X_val_norm)
    print(classification_report(y_val_enc, val_preds, zero_division=0))

    print(" Test Results:")
    test_preds = model.predict(X_test_norm)
    print(classification_report(y_test_enc, test_preds, zero_division=0))

    acc = accuracy_score(y_test_enc, test_preds)
    prec = precision_score(y_test_enc, test_preds, average='weighted', zero_division=0)
    rec = recall_score(y_test_enc, test_preds, average='weighted', zero_division=0)
    f1 = f1_score(y_test_enc, test_preds, average='weighted', zero_division=0)

    print(f" Overall Test Accuracy:  {acc:.4f}")
    print(f" Overall Precision:      {prec:.4f}")
    print(f" Overall Recall:         {rec:.4f}")
    print(f" Overall F1-Score:       {f1:.4f}")

    print(" Confusion Matrix (Test):")
    print(confusion_matrix(y_test_enc, test_preds))

    model_filename = f"{name.replace(' ', '_').lower()}_model.pkl"
    joblib.dump(model, model_filename)
    print(f" Saved model to {model_filename}")
